In [1]:
%cd become-yukarin

/home/ec2-user/bn_ds_challenge/become-yukarin


In [2]:
!pip install --user -r requirements.txt

In [3]:
!pip install --user world4py

In [4]:
import chainer
chainer.print_runtime_info()

Platform: Linux-4.14.173-137.229.amzn2.x86_64-x86_64-with-glibc2.9
Chainer: 5.4.0
NumPy: 1.18.1
CuPy:
  CuPy Version          : 5.4.0
  CUDA Root             : /usr/local/cuda-10.0
  CUDA Build Version    : 10000
  CUDA Driver Version   : 10020
  CUDA Runtime Version  : 10000
  cuDNN Build Version   : 7501
  cuDNN Version         : 7501
  NCCL Build Version    : 2408
  NCCL Runtime Version  : 2408
iDeep: 2.0.0.post3


# 学習
低解像度スペクトログラムを学習データ，高解像度スペクトログラムを教師ラベルとして学習する．  
  - *あらかじめ`config_sr.json`の`input_glob`に`dataset`のパスを書いておく．*  
  - `params.py`の`sample_rate`を`22050`に変更

## `train_sr.py`の引数  
  - `config_json_path`
    - 学習のハイパーパラメータを格納したjsonへのパス
  - `output`
    - 学習した際のハイパーパラメータを格納するjsonを保存するディレクトリ

In [19]:
!PYTHONPATH=`pwd` python train_sr.py '/home/ec2-user/bn_ds_challenge/config_sr.json' '/home/ec2-user/bn_ds_challenge/trained_sr'

predictor/loss
357.315         
181.248         
150.389         
129.622         
140.611         
126.865         
125.001         
120.298         
119.932         
118.765         
116.495         
117.618         
116.335         
119.574         
116.462         
116.36          
115.379         
115.355         
114.29          
113.889         
114.072         
114.95          
113.936         
114.034         
114.116         
112.556         
109.511         
111.489         
110.175         
110.201         
111.651         
110.382         
109.542         
109.956         
108.871         
109.577         
109.228         
108.681         
107.652         
107.386         
106.773         
106.686         
107.12          
107.12          
106.572         
110.092         
106.473         
107.864         
107.908         
105.594         
105.681         
105.205         
106.286         
108.479         
110.629         
106.21          
104.139         
103.991         

In [16]:
!PYTHONPATH=`pwd` python scripts/extract_spectrogram_pair.py -i '/home/ec2-user/bn_ds_challenge/output_wav/raw' -o '/home/ec2-user/bn_ds_challenge/feature' --sample_rate 22050

{'alpha': 0.466,
 'enable_overwrite': False,
 'f0_estimating_method': 'harvest',
 'frame_period': 5,
 'input_directory': PosixPath('/home/ec2-user/bn_ds_challenge/output_wav/raw'),
 'order': 8,
 'output_directory': PosixPath('/home/ec2-user/bn_ds_challenge/feature'),
 'pad_second': 0.0,
 'sample_rate': 22050,
 'top_db': None}
100%|██████████████████████████████████████| 100/100 [00:00<00:00, 51363.02it/s]


# 超解像のテスト
## `super_resolution_test.py`の引数
`test_data_sr`に超解像したいwavを入れておく．
  - `model_names`
    - 学習済みモデル(*.npz)を含むディレクトリ(複数指定可)
      -  今回は`trained_sr/`
  - `-md`, `--model_directory`
    - `model_names`の親ディレクトリ
       - 今回は`/content/drive/My Drive/bn_ds_challenge`
  - `-iwd`, `--input_wave_directory`
    - 低解像度と高解像度のスペクトログラムを生成した変換*先*のwavファイルのディレクトリ
  - `-g`, `--gpu`
    - GPUのid(整数値)

変換結果は`output/trained_sr/`に出力される．

In [0]:
!cp '/content/drive/My Drive/bn_ds_challenge/output_wav/noised_tgt/noised_tgt_001.wav' './become_yukarin/test_data_sr'

# 低解像度と高解像度のスペクトログラムを抽出

wavデータから，低解像度と高解像度のスペクトログラムを生成．このwavは変換*先*のやつ

In [0]:
!PYTHONPATH=`pwd` python scripts/super_resolution_test.py 'trained_sr' \
    -md '/content/drive/My Drive/bn_ds_challenge' \
    -iwd '/content/drive/My Drive/bn_ds_challenge/output_wav/raw' \
    -g 0

In [5]:
from glob import glob
from pathlib import Path

import numpy as np
import librosa

from become_yukarin.config.config import create_from_json as create_config
from become_yukarin.config.sr_config import create_from_json as create_sr_config
from become_yukarin.dataset.dataset import AcousticFeatureProcess
from become_yukarin.dataset.dataset import WaveFileLoadProcess
from become_yukarin.param import Param
from become_yukarin import SuperResolution
from become_yukarin import AcousticConverter

import matplotlib.pyplot as plt
from IPython.display import Audio
%matplotlib inline

In [6]:
model_path = Path('/home/ec2-user/bn_ds_challenge/trained_sr/predictor_35000.npz')
config_path = Path('/home/ec2-user/bn_ds_challenge/trained_sr/config.json')
npy_dir = '/home/ec2-user/bn_ds_challenge/sred_npy/'
noised_dir = '/home/ec2-user/bn_ds_challenge/dist-data/noised_tgt/'

# モデルのパラメータ取得
config = create_sr_config(config_path)
param = config.dataset.param
sr = param.voice_param.sample_rate # サンプリング周波数

In [35]:
def noise_reduction(input_wav_path, model_path=model_path, config_path=config_path, config=config, param=param, sr=sr):
    # wavファイルの読み込みのパラメータ設定
    wave_process = WaveFileLoadProcess(
            sample_rate=param.voice_param.sample_rate,
            top_db=None,
    )
    # 音響特徴量のパラメータ設定
    acoustic_feature_process = AcousticFeatureProcess(
        frame_period=param.acoustic_feature_param.frame_period,
        order=param.acoustic_feature_param.order,
        alpha=param.acoustic_feature_param.alpha,
        f0_estimating_method=param.acoustic_feature_param.f0_estimating_method,
    )
    # 音響特徴量の抽出
    inp = acoustic_feature_process(wave_process(str(input_wav_path)))
    super_resolution = SuperResolution(config, model_path, gpu=0)
    # 超解像
    wave = super_resolution(inp.spectrogram, acoustic_feature=inp, sampling_rate=sr)
    
    out_name = str(input_wav_path.stem).replace('noised_', '')
    
    # メルスペクトログラムに変換
    mel = librosa.feature.melspectrogram(y=wave.wave)
    print('超解像したメルスペクトログラムの次元:', mel.shape)
    
    x = np.load(noised_dir + input_wav_path.stem + '.npy')
    print('変換前の次元:', x.shape)
    np.save(file=npy_dir + out_name + '.npy', arr=mel, allow_pickle=False, fix_imports=False)

    # wavファイル書き出し
    librosa.output.write_wav('/home/ec2-user/bn_ds_challenge/sred_wav/' + out_name + '.wav', wave.wave, wave.sampling_rate)
    print(out_name + '.wav を書き出し完了')

In [36]:
noised_wav_files_list = list(Path('/home/ec2-user/bn_ds_challenge/output_wav/noised_tgt/').glob('*.wav'))

In [37]:
n_path = noised_wav_files_list[1]
print(n_path.name)
Audio(filename=str(n_path))

noised_tgt_009.wav


In [38]:
for file_path in noised_wav_files_list:
    print('{}を超解像中...{}/{}'.format(file_path.name, noised_wav_files_list.index(file_path) + 1, len(noised_wav_files_list)))
    noise_reduction(file_path)

noised_tgt_012.wavを超解像中...1/30
超解像したメルスペクトログラムの次元: (128, 206)
変換前の次元: (128, 206)
tgt_012.wav を書き出し完了
noised_tgt_009.wavを超解像中...2/30
超解像したメルスペクトログラムの次元: (128, 214)
変換前の次元: (128, 214)
tgt_009.wav を書き出し完了
noised_tgt_019.wavを超解像中...3/30
超解像したメルスペクトログラムの次元: (128, 159)
変換前の次元: (128, 159)
tgt_019.wav を書き出し完了
noised_tgt_013.wavを超解像中...4/30
超解像したメルスペクトログラムの次元: (128, 166)
変換前の次元: (128, 166)
tgt_013.wav を書き出し完了
noised_tgt_027.wavを超解像中...5/30
超解像したメルスペクトログラムの次元: (128, 218)
変換前の次元: (128, 218)
tgt_027.wav を書き出し完了
noised_tgt_024.wavを超解像中...6/30
超解像したメルスペクトログラムの次元: (128, 177)
変換前の次元: (128, 177)
tgt_024.wav を書き出し完了
noised_tgt_021.wavを超解像中...7/30
超解像したメルスペクトログラムの次元: (128, 204)
変換前の次元: (128, 204)
tgt_021.wav を書き出し完了
noised_tgt_029.wavを超解像中...8/30
超解像したメルスペクトログラムの次元: (128, 146)
変換前の次元: (128, 146)
tgt_029.wav を書き出し完了
noised_tgt_020.wavを超解像中...9/30
超解像したメルスペクトログラムの次元: (128, 140)
変換前の次元: (128, 140)
tgt_020.wav を書き出し完了
noised_tgt_003.wavを超解像中...10/30
超解像したメルスペクトログラムの次元: (128, 132)
変換前の次元: (128, 132)
tgt_003.w

In [17]:
reduced_wav_files_list = list(Path('/home/ec2-user/bn_ds_challenge/sred_wav/').glob('*.wav'))

In [18]:
r_path = reduced_wav_files_list[1]
print(r_path.name)
Audio(filename=str(r_path))

tgt_009.wav
